In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('google_mt5-xl_1shot_tfidf.csv')
df

,id,dialogue,summary,keywords,processed_dialogue,spans_to_fill,responses
0,13862856,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...,"['needs', 'larry', 'contact']",Summarize the conversation:\nJoseph: I have a ...,<extra_id_0> needs <extra_id_1> larry <extra_i...,<extra_id_0> amanda <extra_id_1> to contact <e...
1,13729565,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...,Eric and Rob are going to watch a stand-up on ...,"['stand', 'youtube', 'eric']",Summarize the conversation:\nSil: maybe we can...,<extra_id_0> stand <extra_id_1> youtube <extra...,<extra_id_0> Rob wants to watch some of Eric's...
2,13680171,"Lenny: Babe, can you help me with something?\r...",Lenny can't decide which trousers to buy. Bob ...,"['lenny', 'trousers', 'bob']","Summarize the conversation:\nKaren: Hey guys, ...",<extra_id_0> lenny <extra_id_1> trousers <extr...,<extra_id_0> bob helps <extra_id_1> to choose ...
3,13729438,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...,"['emma', 'soon', 'let']",Summarize the conversation:\nTheo: Can we meet...,<extra_id_0> emma <extra_id_1> soon <extra_id_...,<extra_id_0> Will and <extra_id_1> are going t...
4,13828600,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju...",Jane is in Warsaw. Ollie and Jane has a party....,"['jane', 'ollie', 'lunch']","Summarize the conversation:\nGus: Heyy, saw yo...",<extra_id_0> jane <extra_id_1> ollie <extra_id...,<extra_id_0> ollie and <extra_id_1> are having...
...,...,...,...,...,...,...,...
15546,13717247,Clair: We've got an answer!\r\nBob: Go on! Tel...,"The answer is ""of course"".",NaN,NaN,NaN,NaN
15547,13716118,Sandra: Does anyone have this book?\r\nSandra:...,Greg is going to bring the book Sandra is look...,NaN,NaN,NaN,NaN
15548,13814059,Molly: I have been doing freelancing started s...,Molly will consider Oliver for her next freela...,NaN,NaN,NaN,NaN
15549,13820959,Lara: For the people that are in Santa Teresa ...,Sarah and George will go for dinner to Amicis ...,NaN,NaN,NaN,NaN


In [3]:
df = df[~df['spans_to_fill'].isna()]

In [4]:
def consolidate_response(row):
    span_to_fill = row['spans_to_fill']
    response = row['responses']
    # split the response by r'<extra_id_\d+>'
    spans_filled = re.split(r'<extra_id_\d+>', response)
    spans_filled = spans_filled[1:]

    # replace span_to_fill with spans_filled
    formatted_response = span_to_fill
    for i in range(len(spans_filled)):
        formatted_response = formatted_response.replace('<extra_id_{}>'.format(i), spans_filled[i], 1)
    return formatted_response

In [5]:
consolidate_responses = df.apply(consolidate_response, axis=1)

In [6]:
# compute the rouge score
from rouge import Rouge

rouge = Rouge()

rouge_scores = rouge.get_scores(consolidate_responses, df['summary'])
rouge_scores

[{'rouge-1': {'r': 0.23076923076923078, 'p': 0.5, 'f': 0.31578946936288094},
  'rouge-2': {'r': 0.15384615384615385,
   'p': 0.3333333333333333,
   'f': 0.21052631146814413},
  'rouge-l': {'r': 0.23076923076923078, 'p': 0.5, 'f': 0.31578946936288094}},
 {'rouge-1': {'r': 0.45454545454545453,
   'p': 0.29411764705882354,
   'f': 0.357142852372449},
  'rouge-2': {'r': 0.2, 'p': 0.125, 'f': 0.1538461491124262},
  'rouge-l': {'r': 0.45454545454545453,
   'p': 0.29411764705882354,
   'f': 0.357142852372449}},
 {'rouge-1': {'r': 0.13636363636363635,
   'p': 0.2727272727272727,
   'f': 0.18181817737373748},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.09090909090909091,
   'p': 0.18181818181818182,
   'f': 0.12121211676767694}},
 {'rouge-1': {'r': 0.7, 'p': 0.4117647058823529, 'f': 0.5185185138545954},
  'rouge-2': {'r': 0.2, 'p': 0.11764705882352941, 'f': 0.14814814348422511},
  'rouge-l': {'r': 0.5, 'p': 0.29411764705882354, 'f': 0.3703703657064472}},
 {'rouge-1': {'r':

In [7]:
df_results = []
for i in range(len(df)):
    session = df.iloc[i]
    response = consolidate_responses[i]
    d = {
            'session_id': session.index,
            'gold_summary': session['summary'],
            'response_summary': response,
            'rouge_1_f': rouge_scores[i]['rouge-1']['f'],
            'rouge_2_f': rouge_scores[i]['rouge-2']['f'],
            'rouge_l_f': rouge_scores[i]['rouge-l']['f'],
            'rouge_1_p': rouge_scores[i]['rouge-1']['p'],
            'rouge_2_p': rouge_scores[i]['rouge-2']['p'],
            'rouge_l_p': rouge_scores[i]['rouge-l']['p'],
            'rouge_1_r': rouge_scores[i]['rouge-1']['r'],
            'rouge_2_r': rouge_scores[i]['rouge-2']['r'],
            'rouge_l_r': rouge_scores[i]['rouge-l']['r'],
            'scores': rouge_scores[i],
            'spans_to_fill': session['spans_to_fill'],
            #         'keywords': session['keywords'],
            #         'missing_keywords': [x for x in session['keywords'] if x not in predictions[i].lower()]
            }
    df_results.append(d)

In [8]:
pd.DataFrame(df_results).to_csv('consolidated.csv')

In [9]:
df = pd.DataFrame(df_results)
df

,session_id,gold_summary,response_summary,rouge_1_f,rouge_2_f,rouge_l_f,rouge_1_p,rouge_2_p,rouge_l_p,rouge_1_r,rouge_2_r,rouge_l_r,scores,spans_to_fill
0,"Index(['id', 'dialogue', 'summary', 'keywords'...",Hannah needs Betty's number but Amanda doesn't...,amanda needs to contact larry Summary: am...,0.315789,0.210526,0.315789,0.500000,0.333333,0.500000,0.230769,0.153846,0.230769,"{'rouge-1': {'r': 0.23076923076923078, 'p': 0....",<extra_id_0> needs <extra_id_1> larry <extra_i...
1,"Index(['id', 'dialogue', 'summary', 'keywords'...",Eric and Rob are going to watch a stand-up on ...,Rob wants to watch some of Eric's stand -ups...,0.357143,0.153846,0.357143,0.294118,0.125000,0.294118,0.454545,0.200000,0.454545,"{'rouge-1': {'r': 0.45454545454545453, 'p': 0....",<extra_id_0> stand <extra_id_1> youtube <extra...
2,"Index(['id', 'dialogue', 'summary', 'keywords'...",Lenny can't decide which trousers to buy. Bob ...,bob helps lenny to choose a pair of purple ...,0.181818,0.000000,0.121212,0.272727,0.000000,0.181818,0.136364,0.000000,0.090909,"{'rouge-1': {'r': 0.13636363636363635, 'p': 0....",<extra_id_0> lenny <extra_id_1> trousers <extr...
3,"Index(['id', 'dialogue', 'summary', 'keywords'...",Emma will be home soon and she will let Will k...,"Will and emma are going to be home soon , ...",0.518519,0.148148,0.370370,0.411765,0.117647,0.294118,0.700000,0.200000,0.500000,"{'rouge-1': {'r': 0.7, 'p': 0.4117647058823529...",<extra_id_0> emma <extra_id_1> soon <extra_id_...
4,"Index(['id', 'dialogue', 'summary', 'keywords'...",Jane is in Warsaw. Ollie and Jane has a party....,ollie and jane are having lunch. ollie an...,0.100000,0.000000,0.100000,0.285714,0.000000,0.285714,0.060606,0.000000,0.060606,"{'rouge-1': {'r': 0.06060606060606061, 'p': 0....",<extra_id_0> jane <extra_id_1> ollie <extra_id...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,"Index(['id', 'dialogue', 'summary', 'keywords'...",Benjamin didn't come to see a basketball game ...,Alex and benjamin are going to a basketbal...,0.382979,0.076923,0.382979,0.600000,0.117647,0.600000,0.281250,0.057143,0.281250,"{'rouge-1': {'r': 0.28125, 'p': 0.6, 'f': 0.38...",<extra_id_0> benjamin <extra_id_1> basketball ...
815,"Index(['id', 'dialogue', 'summary', 'keywords'...",The audition starts at 7.30 P.M. in Antena 3.,Jamilla remembers that the antena 3 auditi...,0.615385,0.416667,0.615385,0.533333,0.357143,0.533333,0.727273,0.500000,0.727273,"{'rouge-1': {'r': 0.7272727272727273, 'p': 0.5...",<extra_id_0> antena <extra_id_1> audition <ext...
816,"Index(['id', 'dialogue', 'summary', 'keywords'...","Marta sent a file accidentally,",<file_photo> <file_photo> <file_photo> < fil...,0.125000,0.000000,0.125000,0.090909,0.000000,0.090909,0.200000,0.000000,0.200000,"{'rouge-1': {'r': 0.2, 'p': 0.0909090909090909...",<extra_id_0> file <extra_id_1> marta <extra_id...
817,"Index(['id', 'dialogue', 'summary', 'keywords'...",There was a meet-and-greet with James Charles ...,British media was surprised that James Charle...,0.451613,0.200000,0.387097,0.388889,0.166667,0.333333,0.538462,0.250000,0.461538,"{'rouge-1': {'r': 0.5384615384615384, 'p': 0.3...",<extra_id_0> gathered <extra_id_1> greet <extr...


In [10]:
print('\n'.join([str(x) for x in df.mean()]), sep='\t')

0.3318115619808207
0.12054343391610574
0.3009858835167095
0.378282360861946
0.13341665814260534
0.3439423741174769
0.32570176793455013
0.12543281422789615
0.2952348966842267


C:\Users\chris\Desktop\CodeRepositories\venv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.
